<a href="https://colab.research.google.com/github/LAKSH3002/Airline-Passenger-Analytics/blob/main/Basic_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
print(pd.__version__)
from google.colab import drive
drive.mount('/content/drive') # Updated mountpoint to a valid local directory

2.2.2
Mounted at /content/drive


In [ ]:
# Reading and operating on csv files
df = pd.read_csv('/content/drive/My Drive/Datasets_for_learning/superhero_movies_dataset.csv')
print(df.to_string())

# max number of rows
print(pd.options.display.max_rows) # 60
# This maximum number is the number of rows that can be displayed through the system
# You can increase by assigning a value to it as well (after this read csv file and print it)
pd.options.display.max_rows = 9999
print(pd.options.display.max_columns) # 20
# print(df2.head()) # Quick overview (you can also specify the number of rows you want, otherwise default is 5)
# print(df2.tail()) # Gives the last rows

print(df.info()) # This gives you exact information that you require from a data set.

     Year                                                              Film                               Publisher                                   Director                                                                                           Distributor                                            Worldwide_Gross                                                                                                                                                                                                      Description
0    1937                                                The Shadow Strikes                          Street & Smith                                Lynn Shores                                                                             Grand National Films Inc.                                                        NaN                                                                                                                                                        

In [ ]:
# Cleaning of data set.
print(df.info())
print(df.isnull().sum()) # It gives you the number of null values

# You can remove null value rows using .dropna() or replace null values using .fillna()

# print(df2['Worldwide_Gross'].unique()) # prints all unique values
# print(df2['Description'].unique()) - prints all unique values
suspicious_values = ["N/A", "n/a", "na", "NA", "null", "NULL", "?", "-", "none", "None", " "]
for val in suspicious_values:
  print(f"col1 - '{val}':", (df['Worldwide_Gross'] == val).sum())
  print(f"col2 - '{val}':", (df['Description'] == val).sum())

# This dataset just has null values (NaN) - NaN is the most common null entry
# Besides that the other ones are suspicious_values mentioned above.

both_null = df[df['Worldwide_Gross'].isna() & df['Description'].isna()]
print(len(both_null))

gross_null = df[df['Worldwide_Gross'].isna() & df['Description'].notna()]
print(len(gross_null))

desc_null = df[df['Worldwide_Gross'].notna() & df['Description'].isna()]
print(len(desc_null))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Year             407 non-null    int64 
 1   Film             407 non-null    object
 2   Publisher        407 non-null    object
 3   Director         407 non-null    object
 4   Distributor      407 non-null    object
 5   Worldwide_Gross  279 non-null    object
 6   Description      295 non-null    object
dtypes: int64(1), object(6)
memory usage: 22.4+ KB
None
Year                 0
Film                 0
Publisher            0
Director             0
Distributor          0
Worldwide_Gross    128
Description        112
dtype: int64
col1 - 'N/A': 0
col2 - 'N/A': 0
col1 - 'n/a': 0
col2 - 'n/a': 0
col1 - 'na': 0
col2 - 'na': 0
col1 - 'NA': 0
col2 - 'NA': 0
col1 - 'null': 0
col2 - 'null': 0
col1 - 'NULL': 0
col2 - 'NULL': 0
col1 - '?': 0
col2 - '?': 0
col1 - '-': 0
col2 - '-': 0
col1 - 'none'

In [ ]:
# Till now we have found out number of null values in the dataset, two columns has them as NaN
# Now we have to find a way to make a proper dataset to carry on visualization and correlation stuff.
# For Gross, convert it into numeric and replace with median, for Description, replace it with description not availaible.

# Column 1
# Clean and convert Worldwide_Gross in one go
print(df['Worldwide_Gross'].isna().sum())
print(df['Description'].isna().sum())

128
112


In [ ]:
import re
import numpy as np
import pandas as pd

def parse_money(x: object) -> float:
    """Return a float from strings like 'Theatrical ($177.2,000,000)' or '($63,000,000.3)'. """
    if x is None:
        return np.nan
    s = str(x).strip()
    if s == "" or s.lower() in {"nan", "none"}:
        return np.nan

    # Prefer the part inside parentheses, if present
    m = re.search(r'\(([^)]*)\)', s)
    if m:
        s = m.group(1)

    # Fix cases like '177.2,000,000' where '.' is used before a comma -> treat '.' as thousands sep
    s = re.sub(r'\.(?=[0-9]{1,3},)', '', s)

    # Keep only digits, commas, and dots; drop $, spaces, words, etc.
    s = re.sub(r'[^0-9\.,]', '', s)

    # Remove thousand commas
    s = s.replace(',', '')

    # If multiple dots remain, they’re thousand dots -> remove all
    if s.count('.') > 1:
        s = s.replace('.', '')

    # If single dot but >3 digits after it, likely not a decimal -> remove it
    if s.count('.') == 1:
        i = s.find('.')
        if len(s) - i - 1 > 3:
            s = s.replace('.', '')

    try:
        return float(s)
    except Exception:
        return np.nan

# Apply the parser
df['Worldwide_Gross'] = df['Worldwide_Gross'].apply(parse_money)

# Fill NaNs with median (only if we have at least one valid number)
if df['Worldwide_Gross'].notna().any():
    median_gross = df['Worldwide_Gross'].median()
    df['Worldwide_Gross'] = df['Worldwide_Gross'].fillna(median_gross)
else:
    # fallback (change 0 to whatever default you prefer)
    df['Worldwide_Gross'] = 0.0

print(df['Worldwide_Gross'].isna().sum())
# print(df.head(20))

# Replacing NaN in description column with No description availaible
df['Description'] = df['Description'].fillna('No description available')
print(df['Description'].isna().sum())

print('Now the new dataset is')
print(df.isnull().sum())

# Save cleaned dataset to CSV
df.to_csv('superhero_movies_dataset_cleaned.csv', index=False)

# Download it to your computer
from google.colab import files
files.download('superhero_movies_dataset_cleaned.csv')

0
0
Now the new dataset is
Year               0
Film               0
Publisher          0
Director           0
Distributor        0
Worldwide_Gross    0
Description        0
dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>